In [ ]:
#loaa function
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCrossEntropyLossWithL21Regularization(nn.Module):
    def __init__(self, weight=None, reduction='mean', lambda_l21=0.001):
        super(CustomCrossEntropyLossWithL21Regularization, self).__init__()
        self.weight = weight
        self.reduction = reduction
        self.lambda_l21 = lambda_l21

    def l2_1_regularization(self, model):
        reg_loss = 0
        for param in model.parameters():
            if param.dim() > 1:
                reg_loss += torch.sum(torch.sqrt(torch.sum(param**2, dim=1))) 
        return reg_loss

    def forward(self, input, target, model=None):

        loss = F.cross_entropy(input, target, weight=self.weight, reduction=self.reduction)

        if model is not None:
            l2_1_loss = self.l2_1_regularization(model)
            loss += self.lambda_l21 * l2_1_loss

        return loss

In [ ]:
#part1 ：Att_net
from math import sqrt
 
import torch
import torch.nn as nn
 
class Att_brainnet(nn.Module):
    def __init__(self, models, device='cpu'):
        super(Att_brainnet, self).__init__()
        
        self.models = models 
        
 
    def forward(self):
        # x: batch, n, dim_q
        X_newRA_IA = torch.tensor(X_newRA_I, dtype=torch.float64)
        X_newEA_IA = torch.tensor(X_newEA_I, dtype=torch.float64)
        
        for lll, model in enumerate(self.models):
            Feature=torch.zeros((NUM_R[lll],161),dtype=torch.float64)
            for i in range(161):
                for j in range(NUM_R[lll]):
                    Feature[j][i] = to_tensor(X_bian_R[lll][j][i])
            YQ = model(Feature)
            tagAA=torch.sum(YQ,dim=1).detach().reshape(-1, 1)        

            mean = tagAA.mean(dim=0, keepdim=True)
            std = tagAA.std(dim=0, keepdim=True)

            tagAAG = (tagAA - mean) / (std + 1e-6)

            tagAAG=tagAAG.reshape(-1, 1)
            for j in range(NUM_R[lll]):
                SS1 = np.array(ROI_record[lll][j],dtype='int')
                SS2 = np.array(list_BEST[SS1].pos_ac,dtype='int')
                X_newRA_IA[:, SS2] = X_newRA_IA[:, SS2] + (X_newR_tensor[:, SS2] * (tagAAG[j] / NA_tensor[SS1]))
                X_newEA_IA[:, SS2] = X_newEA_IA[:, SS2] + (X_newE_tensor[:, SS2] * (tagAAG[j] / NA_tensor[SS1]))
            lll=lll+1
        
        return X_newRA_IA,X_newEA_IA

In [ ]:
models = [SelfAttention(161, NUM_R[i], NUM_R[i], device='cpu') for i in range(8)]

In [ ]:
for lll,model in enumerate(models):
    save_model = torch.load("./SA_FINAL_"+str(op_now)+"_"+str(iii)+"_"+str(lll)+".pth")
    model_dict =  model.state_dict()
    state_dict = {k:v for k,v in save_model.items() if k in model_dict.keys()}
    model_dict.update(state_dict)
    model.load_state_dict(model_dict)
    print(f"Model {lll+1} loaded with weights from {save_model}")

In [ ]:
att_model = Att_brainnet(models=models, device='cpu')

In [ ]:
torch.set_grad_enabled(True)

In [ ]:
params = [model.parameters() for model in models]
all_params = [param for sublist in params for param in sublist]
optimizer = torch.optim.Adam(all_params, lr=0.001)

In [ ]:
Y_train_tensor=torch.from_numpy(Y_train)
Y_train_tensor=Y_train_tensor.long()

In [ ]:
epochs = 10 
for epoch in range(epochs):
    optimizer.zero_grad()
    
    total_loss = 0
    att_model.train()
        
    outputR,outputE = att_model()

    loss = criterion(outputR,Y_train_tensor)
    
    print(outputR,outputE)
        
    X_trainA=outputR.detach().numpy()
    Y_trainA=Y_train
    X_testA=outputE.detach().numpy()
    Y_testA=Y_test
    print(X_trainA.shape)
    print(X_testA.shape)
    clf_linear = svm.SVC(kernel='linear',C=0.095)
    clf_linear.fit(X_trainA,Y_trainA)
    score_linear_test = clf_linear.score(X_testA,Y_testA)
    score_linear_train = clf_linear.score(X_trainA,Y_trainA)
    predict_test = clf_linear.predict(X_testA)
    print(predict_test,Y_testA)
    print(epoch,"SVM Test  Accuracy : %.4g" % (score_linear_test))
    print(epoch,"SVM Train  Accuracy : %.4g" % (score_linear_train))
    
    loss.backward()
    
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{epochs}], Total Loss: {loss.item():.4f}")

In [ ]:
#part1I ：Att_pwn
from math import sqrt
 
import torch
import torch.nn as nn
 
class Att_connect(nn.Module):
    def __init__(self, modelsPT, modelsNT, modelsPE, modelsNE, device='cpu'):
        super(Att_connect, self).__init__()
        
        self.modelsPT = modelsPT
        self.modelsNT = modelsNT
        
        self.modelsPE = modelsPE
        self.modelsNE = modelsNE
        
 
    def forward(self):
        X_newRA_IIA = torch.tensor(X_newRA_II, dtype=torch.float64)
        X_newEA_IIA = torch.tensor(X_newEA_II, dtype=torch.float64)
        
        u=0
        for u, model in enumerate(self.modelsPE):
            tagAA1=torch.zeros(POS_test[u])
            if (POS_test[u]!=0):
                Feature1=torch.zeros((POS_test[u],1),dtype=torch.float64)
                for j in range(POS_test[u]):
                    Feature1[j][0]=to_tensor(X_bian_P_test[j][u])
                YQ1=model(Feature1)
                tagAA1=torch.sum(YQ1,dim=1)
                for j in range(POS_test[u]): 
                    X_newEA_IIA[u][POS_record_test[u][j]] = X_newEA_IIA[u][POS_record_test[u][j]].clone() * (tagAA1[j])
            u=u+1
        
        for u, model in enumerate(self.modelsNE):
            tagAA2=torch.zeros(NEV_test[u])
            if (NEV_test[u]!=0):
                Feature2=torch.zeros((NEV_test[u],1),dtype=torch.float64)
                for j in range(NEV_test[u]):
                    Feature2[j][0]=to_tensor(X_bian_N_test[j][u])
                YQ2=model(Feature2)
                tagAA2=torch.sum(YQ2,dim=1)
                for j in range(NEV_test[u]):
                    X_newEA_IIA[u][NEV_record_test[u][j]]=X_newEA_IIA[u][NEV_record_test[u][j]].clone()*(tagAA2[j])
        
        for v, model in enumerate(self.modelsPT):
            tagAA1=torch.zeros(POS_train[v])
            if (POS_train[v]!=0):
                Feature1=torch.zeros((POS_train[v],1),dtype=torch.float64)
                for j in range(POS_train[v]):
                    Feature1[j][0]=to_tensor(X_bian_P_train[j][v])
                YQ1=model(Feature1)
                tagAA1=torch.sum(YQ1,dim=1)
                for j in range(POS_train[v]): 
                    X_newRA_IIA[v][POS_record_train[v][j]]=X_newRA_IIA[v][POS_record_train[v][j]].clone()*(tagAA1[j])
    
        for v, model in enumerate(self.modelsNT):
            tagAA2=torch.zeros(NEV_train[v])
            if (NEV_train[v]!=0):
                Feature2=torch.zeros((NEV_train[v],1),dtype=torch.float64)
                for j in range(NEV_train[v]):
                    Feature2[j][0]=to_tensor(X_bian_N_train[j][v])
                YQ2=model(Feature2)
                tagAA2=torch.sum(YQ2,dim=1)
                for j in range(NEV_train[v]):
                    X_newRA_IIA[v][NEV_record_train[v][j]]=X_newRA_IIA[v][NEV_record_train[v][j]].clone()*(tagAA2[j])
    
        mean = X_newRA_IIA.mean(dim=0, keepdim=True)
        std  = X_newRA_IIA.std(dim=0, keepdim=True)
        X_newRA_IIA = (X_newRA_IIA - mean) / (std + 1e-6)
            
        mean = X_newEA_IIA.mean(dim=0, keepdim=True)
        std  = X_newEA_IIA.std(dim=0, keepdim=True)
        X_newEA_IIA = (X_newEA_IIA - mean) / (std + 1e-6)
            
        return X_newRA_IIA,X_newEA_IIA

In [ ]:
modelsPE = [SelfAttention(1, POS_test[u], POS_test[u], device='cpu') for u in range(23)]
modelsNE = [SelfAttention(1, NEV_test[u], NEV_test[u], device='cpu') for u in range(23)]

modelsPT = [SelfAttention(1, POS_train[v], POS_train[v], device='cpu') for v in range(138)]
modelsNT = [SelfAttention(1, NEV_train[v], NEV_train[v], device='cpu') for v in range(138)]

In [ ]:
for lll,model in enumerate(modelsPE):
    print(lll)
    save_model = torch.load("./SA_FINAL_"+str(iii)+"_"+str(lll*7)+"_"+str(op_now)+"A.pth")
    model_dict =  model.state_dict()
    state_dict = {k:v for k,v in save_model.items() if k in model_dict.keys()}
    model_dict.update(state_dict)
    model.load_state_dict(model_dict)
    print(f"Model {lll} loaded with weights from {save_model}")

In [ ]:
for lll,model in enumerate(modelsNE):
    print(lll)
    save_model = torch.load("./SA_FINAL_"+str(iii)+"_"+str(lll*7)+"_"+str(op_now)+"B.pth")
    model_dict =  model.state_dict()
    state_dict = {k:v for k,v in save_model.items() if k in model_dict.keys()}
    model_dict.update(state_dict)
    model.load_state_dict(model_dict)
    print(f"Model {lll} loaded with weights from {save_model}")

In [ ]:
LLL=0
for lll,model in enumerate(modelsPT):
    if (lll%6==op_now):
        LLL=LLL+1
    print(lll,LLL)
    save_model = torch.load("./SA_FINAL_"+str(iii)+"_"+str(lll+LLL)+"_"+str(op_now)+"A.pth")
    model_dict =  model.state_dict()
    state_dict = {k:v for k,v in save_model.items() if k in model_dict.keys()}
    model_dict.update(state_dict)
    model.load_state_dict(model_dict)
    print(f"Model {lll+1} loaded with weights from save_model")

In [ ]:
LLL=0
for lll,model in enumerate(modelsNT):
    if (lll%6==op_now):
        LLL=LLL+1
    print(lll,LLL)
    save_model = torch.load("./SA_FINAL_"+str(iii)+"_"+str(lll+LLL)+"_"+str(op_now)+"B.pth")
    model_dict =  model.state_dict()
    state_dict = {k:v for k,v in save_model.items() if k in model_dict.keys()}
    model_dict.update(state_dict)
    model.load_state_dict(model_dict)
    print(f"Model {lll+1} loaded with weights from save_model")

In [ ]:
pnw_model = Att_connect(modelsPE=modelsPE,modelsNE=modelsNE,modelsPT=modelsPT,modelsNT=modelsNT, device='cpu')

In [ ]:
paramsPE = [model.parameters() for model in modelsPE]
paramsNE = [model.parameters() for model in modelsNE]
paramsPT = [model.parameters() for model in modelsPT]
paramsNT = [model.parameters() for model in modelsNT]

all_paramsPE = [param for sublist in paramsPE for param in sublist]
all_paramsNE = [param for sublist in paramsNE for param in sublist]
all_paramsPT = [param for sublist in paramsPT for param in sublist]
all_paramsNT = [param for sublist in paramsNT for param in sublist]

optimizerPE = torch.optim.Adam(all_paramsPE, lr=0.0001)
optimizerNE = torch.optim.Adam(all_paramsNE, lr=0.0001)
optimizerPT = torch.optim.Adam(all_paramsPT, lr=0.0001)
optimizerNT = torch.optim.Adam(all_paramsNT, lr=0.0001)

In [ ]:
Y_train_tensor=torch.from_numpy(Y_train)
Y_train_tensor=Y_train_tensor.long()

In [ ]:
epochs = 10
for epoch in range(epochs):
    optimizerPE.zero_grad() 
    optimizerNE.zero_grad()
    optimizerPT.zero_grad()
    optimizerNT.zero_grad()
    
    total_loss = 0
    pnw_model.train()
        
    outputR,outputE = pnw_model()
        
    loss = criterion(outputR,Y_train_tensor)
    
    print(outputR,outputE)
        
    X_trainA=outputR.detach().numpy()
    Y_trainA=Y_train
    X_testA=outputE.detach().numpy()
    Y_testA=Y_test
    print(X_trainA.shape)
    print(X_testA.shape)
    clf_linear = svm.SVC(kernel='linear',C=0.095)
    clf_linear.fit(X_trainA,Y_trainA)
    score_linear_test = clf_linear.score(X_testA,Y_testA)
    score_linear_train = clf_linear.score(X_trainA,Y_trainA)
    predict_test = clf_linear.predict(X_testA)
    print(predict_test,Y_testA)
    print(epoch,"SVM Test  Accuracy : %.4g" % (score_linear_test))
    print(epoch,"SVM Train  Accuracy : %.4g" % (score_linear_train))
    
    loss.backward()
    
    optimizerPE.step()
    optimizerNE.step()
    optimizerPT.step()
    optimizerNE.step()
    
    print(f"Epoch [{epoch+1}/{epochs}], Total Loss: {loss.item():.4f}")

In [ ]:
#part3 Att_crm
from math import sqrt
 
import torch
import torch.nn as nn
 
class Att_cross(nn.Module):
    def __init__(self, modelsC, device='cpu'):
        super(Att_cross, self).__init__()
        
        self.modelsC = modelsC 
 
    def forward(self):

        X_newRA_IIIA = torch.tensor(X_newRA_III, dtype=torch.float64)
        X_newEA_IIIA = torch.tensor(X_newEA_III, dtype=torch.float64)
        
        Feature3=torch.zeros((iii+7,161),dtype=torch.float64)
        
        for i in range(iii+7):
            u,v=0,0;
            for j in range(161):
                if (j%7==op_now):
                    Feature3[i][j]=to_tensor(X_newEA_IIIA[u][i])
                    u=u+1
                else:
                    Feature3[i][j]=to_tensor(X_newRA_IIIA[v][i])
                    v=v+1
        YQ1=self.modelsC(Feature3)
        tagAA3=torch.sum(YQ1, dim=1)
    
        u,v=0,0;
        for i in range(161):
            if (i%7==op_now):
                for j in range(iii+7):
                    X_newEA_IIIA[u][j]=X_newEA_IIIA[u][j].clone()*tagAA3[j]
                u=u+1
            else:
                for j in range(iii+7):
                    X_newRA_IIIA[v][j]=X_newRA_IIIA[v][j].clone()*tagAA3[j]
                v=v+1
        
        return X_newRA_IIIA,X_newEA_IIIA

In [ ]:
model3=SelfAttention(161, iii+7, iii+7, device='cpu')
save_model = torch.load("./ARZE"+str(iii)+"P"+str(op_now)+".pth")
model_dict = model3.state_dict()
state_dict = {k:v for k,v in save_model.items() if k in model_dict.keys()}
model_dict.update(state_dict)
model3.load_state_dict(model_dict)
print(model_dict)

In [ ]:
att_cross = Att_cross(modelsC=model3)

In [ ]:
optimizer = torch.optim.Adam(att_cross.parameters(), lr=0.0001)

In [ ]:
Y_train_tensor=torch.from_numpy(Y_train)
Y_train_tensor=Y_train_tensor.long()

In [ ]:
epochs = 10
for epoch in range(epochs):
    optimizer.zero_grad()
    
    total_loss = 0
    att_cross.train()

    outputR,outputE = att_cross()
        
    loss = criterion(outputR,Y_train_tensor)
    
    print(outputR,outputE)
        
    X_trainA=outputR.detach().numpy()
    Y_trainA=Y_train
    X_testA=outputE.detach().numpy()
    Y_testA=Y_test
    print(X_trainA.shape)
    print(X_testA.shape)
    clf_linear = svm.SVC(kernel='linear',C=0.095)
    clf_linear.fit(X_trainA,Y_trainA)
    score_linear_test = clf_linear.score(X_testA,Y_testA)
    score_linear_train = clf_linear.score(X_trainA,Y_trainA)
    predict_test = clf_linear.predict(X_testA)
    print(predict_test,Y_testA)
    print(epoch,"SVM Test  Accuracy : %.4g" % (score_linear_test))
    print(epoch,"SVM Train  Accuracy : %.4g" % (score_linear_train))
    
    loss.backward()
    
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{epochs}], Total Loss: {loss.item():.4f}")